In [ ]:
var path = @"D:\Comp\Handwriting\yolov5\runs\detect\exp14\labels\";

In [ ]:
using System.IO;
using System;
using System.Collections.Generic;

In [ ]:
var originalfn = Directory.GetFiles(@"D:\Comp\Handwriting\SmallTest");
var labels = Directory.GetFiles(path);

In [ ]:
var output = "Id,Predicted\n";

In [ ]:
string Run(string labelPath) {
    var suboutput = "";
    var numbers = File.ReadAllText(labelPath).Split("\n");
    var Values = numbers.AsEnumerable().Select<string,(int,double)?>(x => {
        if (x.Trim() == "") return null;
        var split = x.Split(" ");
        try {
            var Class = Convert.ToInt32(split[0]);
            var Position = Convert.ToDouble(split[1]);
            return (Class, Position);
        } catch (Exception) {
            Console.WriteLine($"Class = {split[0]}");
            Console.WriteLine($"Position = {split[1]}");
            throw;
        }
    });

    var list = new SortedList<string,int>();
    foreach (var Value in Values) {
        if (!Value.HasValue) continue;

        var (Class,Position) = Value.Value;
        var PositionString = Position.ToString();
        
        while (list.Keys.Any(x => x == PositionString))
            PositionString += "1";

        list.Add(PositionString, Class);
    }

    if (list.Count == 0) return "";
    suboutput += $"{new FileInfo(labelPath).Name.Split(".")[0]},";

    foreach (var Value in list.Values) {
        suboutput += Value.ToString();
    }
    return suboutput;
}

In [ ]:
static class Extension {
    public static IEnumerable<(int, T)> Enumerate<T>(IEnumerable<T> items) {
        int i = 0;
        foreach (var item in items) yield return (i++, item);
    }
}

In [ ]:
output += string.Join("\n",(labels).Select(x => Run(x)));

In [ ]:
output += "\n";

var sample = File.ReadAllLines("./sample_submission.csv");

for (var i = 0; i < sample.Length; i++) {
    if (!output.Contains(sample[i].Split(",")[0]))
        output += sample[i] + "\n";
}

In [ ]:
File.WriteAllText("./file.csv",output);